In [1]:
if (!requireNamespace("BiocManager", quietly = TRUE))
    install.packages("BiocManager")

BiocManager::install("org.Hs.eg.db")

'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories", package = "BiocManager")' for details.
Replacement repositories:
    CRAN: https://cran.r-project.org

Bioconductor version 3.20 (BiocManager 1.30.27), R 4.4.3 (2025-02-28)

Warning message:
“package(s) not installed when version(s) same as or greater than current; use
  `force = TRUE` to re-install: 'org.Hs.eg.db'”
Old packages: 'base64enc', 'BH', 'BiocParallel', 'Biostrings',
  'clusterProfiler', 'data.table', 'dbplyr', 'DOSE', 'dplyr', 'dtplyr',
  'enrichplot', 'fgsea', 'GenomeInfoDb', 'ggplot2', 'httr', 'igraph',
  'IRanges', 'lattice', 'lubridate', 'MatrixGenerics', 'RSQLite',
  'SparseArray', 'viridisLite', 'vroom', 'yulab.utils'



In [2]:
library(DESeq2)
library(tximport)
library(org.Hs.eg.db)
library(AnnotationDbi)

Loading required package: S4Vectors

Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce, rownames, sapply, saveRDS, setdiff,
    table, tapply, union, unique, unsplit, which.max, which.min



Attaching package: ‘S4Vectors’


The following object is masked from ‘package:utils’:

    findMatches


The following objects are masked from ‘package:base’:

    expand.grid, I, unname


Loading required package: IRanges

Loading required package: GenomicRanges

Loading required package: GenomeInfoDb



# 1- Read data 

In [3]:
gene_counts <- readRDS("~/genomics//ch2_project/counts/tximport.rds")
 
sample_info <- read.table("~/genomics/ch2_project/counts/sample_info.tsv", header = TRUE, sep = "\t")

# Set factor levels (WT as reference)
sample_info$condition <- factor(sample_info$condition, levels = c("WT", "KO"))

cat(paste0("Samples: ", nrow(sample_info), "\n"))
cat(paste0("Genes: ", nrow(gene_counts$counts), "\n"))
cat(paste0("Conditions: ", paste(levels(sample_info$condition), collapse = " vs "), "\n"))

Samples: 6
Genes: 2379
Conditions: WT vs KO


# 2- Create DESeq dataset 

In [5]:
dds <- DESeqDataSetFromTximport(gene_counts, colData = sample_info, design = ~ condition)

# Pre-filtering: Remove genes with very low counts (genes with less than 10 counts in 3 or more samples will be removed)
keep <- rowSums(counts(dds) >= 10) >= 3 
dds <- dds[keep, ]
cat(paste0("Genes after filtering (>=10 counts in >=3 samples): ", nrow(dds), "\n"))

using counts and average transcript lengths from tximport



Genes after filtering (>=10 counts in >=3 samples): 922


# 3- Run DESeq 

In [6]:
dds <- DESeq(dds)
                res <- results(dds, contrast = c("condition", "KO", "WT"))
res <- res[order(res$padj), ]

summary(res)

# Significant: padj < 0.05 and |log2FC| > 1
sig_genes <- subset(res, padj < 0.05 & abs(log2FoldChange) > 1)
sig_genes <- sig_genes[order(sig_genes$padj), ]

cat(paste0("Total significant genes: ", nrow(sig_genes), "\n"))
cat(paste0("Upregulated in KO: ", sum(sig_genes$log2FoldChange > 0, na.rm = TRUE), "\n"))
cat(paste0("Downregulated in KO: ", sum(sig_genes$log2FoldChange < 0, na.rm = TRUE), "\n"))

estimating size factors

using 'avgTxLength' from assays(dds), correcting for library size

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing




out of 922 with nonzero total read count
adjusted p-value < 0.1
LFC > 0 (up)       : 163, 18%
LFC < 0 (down)     : 144, 16%
outliers [1]       : 0, 0%
low counts [2]     : 0, 0%
(mean count < 9)
[1] see 'cooksCutoff' argument of ?results
[2] see 'independentFiltering' argument of ?results

Total significant genes: 41
Upregulated in KO: 32
Downregulated in KO: 9


# 4- Save data 

In [8]:
dir.create("~/genomics/ch2_project/results/tables", showWarnings = FALSE, recursive = TRUE)

# All results
all_res <- as.data.frame(res)
all_res$gene_id <- rownames(all_res)
all_res <- all_res[, c("gene_id", "baseMean", "log2FoldChange", "lfcSE", "stat", "pvalue", "padj")]
write.csv(all_res, "~/genomics/ch2_project/results/tables/deseq2_all_results.csv", row.names = FALSE)

# Significant genes only
sig_res <- as.data.frame(sig_genes)
sig_res$gene_id <- rownames(sig_res)
sig_res <- sig_res[, c("gene_id", "baseMean", "log2FoldChange", "lfcSE", "stat", "pvalue", "padj")]
write.csv(sig_res, "~/genomics//ch2_project/results/tables/deseq2_significant.csv", row.names = FALSE)

# Normalized counts
norm_counts <- as.data.frame(counts(dds, normalized = TRUE))
norm_counts$gene_id <- rownames(norm_counts)
write.csv(norm_counts, "~/genomics/ch2_project/results/tables/normalized_counts.csv", row.names = FALSE)

# Save DESeq2 object for later use
saveRDS(dds, "~/genomics/ch2_project/results/deseq2_object.rds")

In [10]:
# Function to map Ensembl IDs → Gene Symbols
convert_ensembl_to_symbol <- function(ids) {
  clean_ids <- gsub("\\..*", "", ids)  # remove version numbers if any 
  symbols <- mapIds(org.Hs.eg.db,
                    keys = clean_ids,
                    column = "SYMBOL",
                    keytype = "ENSEMBL",
                    multiVals = "first")
  # Replace NA with original Ensembl ID
  symbols[is.na(symbols)] <- clean_ids[is.na(symbols)]
  return(symbols)
}

# Map gene symbols for results
all_res$gene_symbol <- convert_ensembl_to_symbol(all_res$gene_id)
sig_res$gene_symbol <- convert_ensembl_to_symbol(sig_res$gene_id)
norm_counts$gene_symbol <- convert_ensembl_to_symbol(norm_counts$gene_id)

# move gene_symbol column to first position
all_res <- all_res[, c("gene_symbol", setdiff(names(all_res), "gene_symbol"))]
sig_res <- sig_res[, c("gene_symbol", setdiff(names(sig_res), "gene_symbol"))]
norm_counts <- norm_counts[, c("gene_symbol", setdiff(names(norm_counts), "gene_symbol"))]

all_res <- all_res[, setdiff(names(all_res), "gene_id")]
sig_res <- sig_res[, setdiff(names(sig_res), "gene_id")]
norm_counts <- norm_counts[, setdiff(names(norm_counts), "gene_id")]

# Save tables
write.csv(all_res, "~/genomics/ch2_project/results/tables/deseq2_all_results_genesymbol.csv", row.names = FALSE)
write.csv(sig_res, "~/genomics/ch2_project/results/tables/deseq2_significant_genesymbol.csv", row.names = FALSE)
write.csv(norm_counts, "~/genomics/ch2_project/results/tables/normalized_counts_genesymbol.csv", row.names = FALSE)


'select()' returned 1:many mapping between keys and columns

'select()' returned 1:1 mapping between keys and columns

'select()' returned 1:many mapping between keys and columns

